## 위키피디아는 명확한 키워드로 검색해야 관련된 문서를 가져옴

In [11]:
import wikipedia
import pandas as pd
import requests

# MediaWiki API를 이용해 최신 수정 날짜(마지막 수정 날짜) 가져오는 함수
def get_latest_modification_date(page_id):
    api_url = (
        f"https://ko.wikipedia.org/w/api.php?"
        f"action=query&prop=revisions&rvprop=timestamp&rvlimit=1&rvdir=older&pageids={page_id}&format=json"
    )
    response = requests.get(api_url)
    json_data = response.json()
    try:
        latest_mod_date = json_data['query']['pages'][str(page_id)]['revisions'][0]['timestamp']
    except (KeyError, IndexError):
        latest_mod_date = "알 수 없음"
    return latest_mod_date

# 1. 언어 설정 (한국어)
wikipedia.set_lang('ko')

# 2. 사용자 입력 키워드 3개 (예시)
user_keywords = ["폐암", "암 4기", "암 완치"]

# 3. 각 키워드 별로 페이지 정보 가져오기
data_all = []
for keyword in user_keywords:
    try:
        page = wikipedia.page(keyword)
        latest_mod_date = get_latest_modification_date(page.pageid)
        
        data_all.append({
            "사용자 입력 키워드": keyword,
            "페이지 제목": page.title,
            "URL (출처)": page.url,
            "본문 내용": page.content,
            "최신 수정 날짜": latest_mod_date
        })
    except wikipedia.DisambiguationError as e:
        data_all.append({
            "사용자 입력 키워드": keyword,
            "페이지 제목": "모호함",
            "URL (출처)": "모호함",
            "본문 내용": "모호한 키워드: " + ", ".join(e.options[:3]) + "...",
            "최신 수정 날짜": "N/A"
        })
    except wikipedia.PageError:
        data_all.append({
            "사용자 입력 키워드": keyword,
            "페이지 제목": "페이지 없음",
            "URL (출처)": "페이지 없음",
            "본문 내용": "해당 페이지를 찾을 수 없습니다.",
            "최신 수정 날짜": "N/A"
        })
    except Exception as e:
        data_all.append({
            "사용자 입력 키워드": keyword,
            "페이지 제목": "오류",
            "URL (출처)": "오류",
            "본문 내용": f"오류: {e}",
            "최신 수정 날짜": "N/A"
        })

# 4. DataFrame으로 정리 및 깔끔하게 출력
df = pd.DataFrame(data_all)
print("\n--- 사용자 입력 키워드 기반 페이지 정보 DataFrame ---")
df["본문 내용"][0]


--- 사용자 입력 키워드 기반 페이지 정보 DataFrame ---


'폐암(肺癌, 영어: lung cancer, 의학: lungcarcinoma, 문화어: 폐종양)은 허파에 발생하는 암이다. 허파암이라고도 부른다. 폐암은 기도 내 세포에 DNA 손상이 생긴 경우 발생하며, 종종 흡연이나 해로운 화학 물질을 흡입한 경우 발생한다. 손상된 기도 세포는 조절 능력 없이 증식하여 종양으로 성장한다. 이 때 치료하지 않을 경우, 종양은 폐 전체로 퍼져 폐 기능을 손상시킨다. 결국 폐 종양은 전이되어 신체의 다른 부위로 퍼지게 된다. \n폐암 초기는 보통 증상이 없으며, 촬영을 통해서만 발견될 수 있다. 암이 진행되면서 대부분의 사람들은 기침, 호흡곤란 또는 흉통을 경험한다. 다른 증상들은 종양의 위치와 크기에 따라 달라진다. 일반적으로 폐암이 의심되는 환자들은 종양의 위치와 크기를 확인하기 위해 촬영을 받는다. 이후 병리학자가 의심 종양을 조직검사해 현미경으로 확진한 후 최종 진단된다. 병리학자는 또한 기원 세포 종류에 따라 종양을 분류할 수 있다. 폐암의 약 15%는 소세포폐암이며, 나머지 85%는 비소세포폐암으로 선암종, 편평세포암종, 대세포암종이 이에 포함된다. 진단 후, 암이 얼마나 퍼졌는지에 따라 암의 병기를 결정하기 위한 촬영과 조직검사가 추가된다. \n폐암 초기의 치료 방법으로 종양을 제거하는 수술적 방법이 포함되며, 이후 경우에 따라 남은 암세포를 죽이기 위해 방사선 치료와 항암 화학요법이 뒤따른다. 후기 단계의 암은 특정 암 아형을 표적하는 약물 치료와 함께 방사선요법, 항암화학요법으로 치료한다. 그러나 치료를 하더라도 진단 후 5년 생존율은 20%에 불과하다. 생존율은 초기에 진단된 경우, 낮은 연령에서 더 높으며, 남성에 비해 여성의 생존율이 더 높다.    \n대부분의 폐암은 흡연으로 인해 발생한다. 그 이외 발생원인은 석면, 라돈가스 등의 유해 물질 노출, 유전적 돌연변이가 발생한 경우이다. 따라서 폐암 예방을 위해 사람들이 유해 화학물질에 대한 노출을 피하고 금연하도록 장려해야 한다. 금연은 폐암 발병 가능성을

In [12]:
# csv로 저장
df.to_csv("wikipedia_page_info.csv", index=False, encoding="utf-8-sig")
print("CSV 파일로 저장 완료: wikipedia_page_info.csv")

CSV 파일로 저장 완료: wikipedia_page_info.csv
